In [1]:
import pandas as pd
import numpy as np
import csv
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import chi2
import re
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix

In [2]:
df_test = pd.read_csv("test.csv")
df_test.head()

,PRODUCT_ID,TITLE,BULLET_POINTS,DESCRIPTION,PRODUCT_TYPE_ID
0,604373,Manuel d'Héliogravure Et de Photogravure En Re...,NaN,NaN,6142
1,1729783,DCGARING Microfiber Throw Blanket Warm Fuzzy P...,[QUALITY GUARANTEED: Luxury cozy plush polyest...,<b>DCGARING Throw Blanket</b><br><br> <b>Size ...,1622
2,1871949,I-Match Auto Parts Front License Plate Bracket...,"[Front License Plate Bracket Made Of Plastic,D...",Replacement for The Following Vehicles:2020 LE...,7540
3,1107571,PinMart Gold Plated Excellence in Service 1 Ye...,[Available as a single item or bulk packed. Se...,Our Excellence in Service Lapel Pins feature a...,12442
4,624253,"Visual Mathematics, Illustrated by the TI-92 a...",NaN,NaN,6318


In [3]:
len(df_test)

734736

In [4]:
df_train = pd.read_csv("train.csv")
df_train = df_train.dropna()
df_train.head()

,PRODUCT_ID,TITLE,BULLET_POINTS,DESCRIPTION,PRODUCT_TYPE_ID,PRODUCT_LENGTH
2,2765088,PRIKNIK Horn Red Electric Air Horn Compressor ...,"[Loud Dual Tone Trumpet Horn, Compatible With ...","Specifications: Color: Red, Material: Aluminiu...",7537,748.031495
3,1594019,ALISHAH Women's Cotton Ankle Length Leggings C...,[Made By 95%cotton and 5% Lycra which gives yo...,AISHAH Women's Lycra Cotton Ankel Leggings. Br...,2996,787.401574
5,2152929,HINS Metal Bucket Shape Plant Pot for Indoor &...,"[Simple and elegant, great for displaying indo...",HINS Brings you the most Elegant Looking Pot w...,5725,950.000000
7,2026580,Delavala Self Adhesive Kitchen Backsplash Wall...,[HIGH QUALITY PVC MATERIAL: The kitchen alumin...,<p><strong>Aluminum Foil Stickers-good kitchen...,6030,984.251967
9,2998633,Hexwell Essential oil for Home Fragrance Oil A...,[100% Pure And Natural Essential Oil Or Fragra...,"Transform your home, workplace or hotel room i...",8201,393.700787


In [5]:
len(df_train)

1038460

In [6]:
punctuation_signs = list("?:!.,;")

In [7]:
df_train['TITLE'] = df_train['TITLE'].str.replace("\r", " ")
df_train['TITLE'] = df_train['TITLE'].str.replace("\n", " ")
df_train['TITLE'] = df_train['TITLE'].str.replace("    ", " ")
df_train['TITLE'] = df_train['TITLE'].str.replace('"', '')
df_train['TITLE'] = df_train['TITLE'].str.lower()
for punct_sign in punctuation_signs:
  df_train['TITLE'] = df_train['TITLE'].str.replace(punct_sign, '')
df_train['TITLE'] = df_train['TITLE'].str.replace("'s", "")

C:\Users\niles\AppData\Local\Temp\ipykernel_12336\1748697097.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_train['TITLE'] = df_train['TITLE'].str.replace(punct_sign, '')


In [8]:
final_cols = ["TITLE", "PRODUCT_LENGTH"]
df_train = df_train[final_cols]
df_train = df_train.iloc[:10000, :]

In [9]:
df_test['TITLE'] = df_test['TITLE'].str.replace("\r", " ")
df_test['TITLE'] = df_test['TITLE'].str.replace("\n", " ")
df_test['TITLE'] = df_test['TITLE'].str.replace("    ", " ")
df_test['TITLE'] = df_test['TITLE'].str.replace('"', '')
df_test['TITLE'] = df_test['TITLE'].str.lower()
for punct_sign in punctuation_signs:
  df_test['TITLE'] = df_test['TITLE'].str.replace(punct_sign, '')
df_test['TITLE'] = df_test['TITLE'].str.replace("'s", "")

C:\Users\niles\AppData\Local\Temp\ipykernel_12336\4155834508.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_test['TITLE'] = df_test['TITLE'].str.replace(punct_sign, '')


In [10]:
final_cols = ["TITLE", "PRODUCT_ID"]
df_test = df_test[final_cols]
print(len(df_test),len(df_train))

734736 10000


In [11]:
df_train.isna().sum()

TITLE             0
PRODUCT_LENGTH    0
dtype: int64

In [12]:
df_test["TITLE"].fillna("No Data", inplace = True)

In [13]:
df_test.isna().sum()


TITLE         0
PRODUCT_ID    0
dtype: int64

In [14]:
X_train, X_test, y_train = df_train["TITLE"], df_test["TITLE"], df_train["PRODUCT_LENGTH"]


In [15]:
tfidf = TfidfVectorizer(stop_words='english', ngram_range=(1,2), min_df=5)
print('1')
X_train_vectors_tfidf = tfidf.fit_transform(X_train)
print(X_train_vectors_tfidf.shape)
print('1')
X_test_vectors_tfidf = tfidf.transform(X_test)
print(X_test_vectors_tfidf.shape)

1
(10000, 5838)
1
(734736, 5838)


In [16]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn import utils

encoded = preprocessing.LabelEncoder()
encoded_target= encoded.fit_transform(y_train)
lr_tfidf=LogisticRegression(solver = 'liblinear', C=10, penalty = 'l2')
lr_tfidf.fit(X_train_vectors_tfidf, encoded_target)


LogisticRegression(C=10, solver='liblinear')

In [17]:
y_predict = lr_tfidf.predict(X_test_vectors_tfidf)

In [18]:
y_predict

array([ 318, 1085,  694, ...,   88,  694,  417], dtype=int64)

In [19]:
len(y_predict)

734736

In [20]:
d = {"PRODUCT_LENGTH" : y_predict}

In [21]:
df1 = pd.DataFrame(data=d)
df1

,PRODUCT_LENGTH
0,318
1,1085
2,694
3,88
4,290
...,...
734731,353
734732,290
734733,88
734734,694


In [22]:
df1.describe()

,PRODUCT_LENGTH
count,734736.000000
mean,450.819304
std,198.991569
min,0.000000
25%,290.000000
50%,410.000000
75%,608.000000
max,1237.000000


In [23]:
df_test.describe()

,PRODUCT_ID
count,7.347360e+05
mean,1.493725e+06
std,8.669773e+05
min,0.000000e+00
25%,7.396735e+05
50%,1.492776e+06
75%,2.242406e+06
max,2.999998e+06


In [24]:
df_new = pd.concat([df_test, df1], axis = 1)
df_new.head()

,TITLE,PRODUCT_ID,PRODUCT_LENGTH
0,manuel d'héliogravure et de photogravure en re...,604373,318
1,dcgaring microfiber throw blanket warm fuzzy p...,1729783,1085
2,i-match auto parts front license plate bracket...,1871949,694
3,pinmart gold plated excellence in service 1 ye...,1107571,88
4,visual mathematics illustrated by the ti-92 an...,624253,290


In [26]:
l = ["PRODUCT_ID", "PRODUCT_LENGTH"]
df_new = df_new[l]
df_new

,PRODUCT_ID,PRODUCT_LENGTH
0,604373,318
1,1729783,1085
2,1871949,694
3,1107571,88
4,624253,290
...,...,...
734731,921419,353
734732,2456362,290
734733,841529,88
734734,1190194,694


In [27]:
df_new.to_csv('submission2.csv', index=False, header=True)